In [ ]:
from brian2 import *
import csv
import random
import os, sys
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
from scipy.spatial import distance

In [ ]:
min_range_V = 5
max_range_V = 10

In [ ]:
list_attack_generation = ["FLO", "JAM", "SCA", "FOR", "SPO", "SYB", "SIN", "NON"]

In [ ]:
stats_dir = "results/"
plots_dir = "plots/"

### Methods for csv export

In [ ]:
def append_to_csv_file(filename, line):
    # Update the attack file
    with open(filename, 'a') as csvFile:
        writer = csv.writer(csvFile, delimiter=';')
        writer.writerow(line)

def dump_simulation_data_to_csv(attack, test, position_attack, n_attacks, n_neurons, attacked_neurons, coord_attack, stim_value, n_exec, vIncrement, paramI, monitor, filename):
    mon_trains = monitor.spike_trains()
    
    for neuron in mon_trains.keys():
        for time_delta in mon_trains[neuron]:
            append_to_csv_file(filename, [attack, test, position_attack, n_attacks, n_neurons, attacked_neurons, coord_attack, stim_value, n_exec, str(vIncrement), str(paramI), round(time_delta/ms, 1), neuron])

def list_neurons_to_string(list_neurons):
    result = ""
    for neuron in list_neurons:
        result += str(neuron)+"-"
        
    return result[:-1]

### General variables and constants

In [ ]:
list_coordinates_optimal_path = [[0,0], [1,0], [1,1], [1,2], [0,2], [0,3], [0,4], [0,5], [1,5], [2,5], 
                                 [2,4], [2,3], [3,3], [3,2], [3,1], [3,0], [4,0], [5,0], [6,0], [6,1], 
                                 [6,2], [5,2], [5,3], [5,4], [5,5], [6,5], [6,6]]

In [ ]:
STEP_TIME = 1000*ms

# STEP_TIME per movement. As there are 27 positions -> 27*STEP_TIME. 
SIMULATION_TIME = (STEP_TIME/ms*len(list_coordinates_optimal_path))*ms

In [ ]:
SIMULATION_TIME

In [ ]:
# Duration of simulation, after STEP_TIME. Divided by the number of neurons
def get_time_steps_sequential(tAttack):
    return trunc(((SIMULATION_TIME-tAttack)/200)/ms)*ms

def get_number_attacks_per_position_sequential(tAttack):
    return STEP_TIME/get_time_steps_sequential(tAttack)

def get_last_instant_attack_sequential(tAttack):
    return tAttack+get_time_steps_sequential(tAttack)*200

In [ ]:
get_time_steps_sequential(50*ms)

In [ ]:
get_number_attacks_per_position_sequential(50*ms)

In [ ]:
get_last_instant_attack_sequential(50*ms)

### Auxiliary methods to translate between coordinates and indexes

In [ ]:
def generate_list_random_neurons(nNeurons):
    list_neurons = list(range(0, 200))
    result = []

    for i in range(0, nNeurons):
        index = randint(0, len(list_neurons))
        
        result.append(list_neurons[index])
        del list_neurons[index]
    
    return sorted(result)

In [ ]:
# Dict that stores the index value of a neuron: 3D coordinate -> numeric index (0-199)
dict_neurons_to_numbers = {}

counter = 0

for i in range(0, 5):
    for j in range(0, 5): 
        for k in range(0, 8): 
            dict_neurons_to_numbers[(i,j,k)] = counter
            counter+=1

In [ ]:
#dict_neurons_to_numbers

In [ ]:
# Dict that stores the instant in which the mouse is in eah position of the optimal path

dict_instant_optimal_path = {}

counter = 0

for coord in list_coordinates_optimal_path:
    dict_instant_optimal_path[(coord[0],coord[1])]= counter
    counter += STEP_TIME/ms     

# List of the instants previously calculated (list values of dict)
list_values = list(dict_instant_optimal_path.values())
list_instant_optimal_path = []

for value in list_values:
    list_instant_optimal_path.append(round(value))

In [ ]:
def get_visible_coordinates_by_position(x,y):
    
    coords = [[x-1,y-1], [x-1, y], [x-1,y+1], [x,y-1], [x,y], [x,y+1], [x+1,y-1], [x+1,y], [x+1,y+1]]
    result = []
    
    for n in coords:
        if (0 <= n[0] <= 6) and (0 <= n[1] <= 6):
            result.append(n)
            
    return result

In [ ]:
#get_visible_coordinates_by_position(3,3)

In [ ]:
# Get the neurons indexes that are related to a given maze coordinate
def get_neurons_indexes_by_position(x, y):
    
    coords = [
        [x-2,y-2,0], [x-2,y-2,1], [x-2,y-2,2], [x-2,y-2,3], [x-2,y-2,4], [x-2,y-2,5], [x-2,y-2,6], [x-2,y-2,7],
        [x-2,y-1,0], [x-2,y-1,1], [x-2,y-1,2], [x-2,y-1,3], [x-2,y-1,4], [x-2,y-1,5], [x-2,y-1,6], [x-2,y-1,7],
        [x-2,y,0], [x-2,y,1], [x-2,y,2], [x-2,y,3], [x-2,y,4], [x-2,y,5], [x-2,y,6], [x-2,y,7],
        [x-1,y-2,0], [x-1,y-2,1], [x-1,y-2,2], [x-1,y-2,3], [x-1,y-2,4], [x-1,y-2,5], [x-1,y-2,6], [x-1,y-2,7],
        [x-1,y-1,0], [x-1,y-1,1], [x-1,y-1,2], [x-1,y-1,3], [x-1,y-1,4], [x-1,y-1,5], [x-1,y-1,6], [x-1,y-1,7],
        [x-1,y,0], [x-1,y,1], [x-1,y,2], [x-1,y,3], [x-1,y,4], [x-1,y,5], [x-1,y,6], [x-1,y,7],
        [x,y-2,0], [x,y-2,1], [x,y-2,2], [x,y-2,3], [x,y-2,4], [x,y-2,5], [x,y-2,6], [x,y-2,7],
        [x,y-1,0], [x,y-1,1], [x,y-1,2], [x,y-1,3], [x,y-1,4], [x,y-1,5], [x,y-1,6], [x,y-1,7],
        [x,y,0], [x,y,1], [x,y,2], [x,y,3], [x,y,4], [x,y,5], [x,y,6], [x,y,7],
    ]
    
    coord_result = []
    result = []
    
    for n in coords:
        if (0 <= n[0] <= 4) and (0 <= n[1] <= 4):
            coord_result.append(n)
            
    for coord in coord_result:
        result.append(dict_neurons_to_numbers[(coord[0],coord[1],coord[2])])
            
    return result

In [ ]:
#get_neurons_indexes_by_position(0,0)

In [ ]:
def get_related_neurons_visible_positions(x, y):
    # Get the positions visible from the current position
    visible_coords = get_visible_coordinates_by_position(x, y)
    
    # Get the complete list of neurons related to all visible positions
    list_neurons_index = []

    for coord in visible_coords:
        neurons = get_neurons_indexes_by_position(coord[0], coord[1])
        
        for n in neurons:
            list_neurons_index.append(n)
    
    # Remove duplicates in list_neurons_index
    result = []
    
    for n in list_neurons_index:
        if n not in result:
            result.append(n)
            
    return result

In [ ]:
#get_related_neurons_visible_positions(0,2)

In [ ]:
# Get all neurons related to all positions, in a dict. Both list of neurons and count
dict_all_neurons = {}
dict_all_neurons_count = {}

counter = 0

for pos in list_coordinates_optimal_path:
    dict_all_neurons[counter] = get_related_neurons_visible_positions(pos[0], pos[1])
    dict_all_neurons_count[counter] = len(get_related_neurons_visible_positions(pos[0], pos[1]))
    
    counter+=1

### Auxiliary methods for Brian Simulation

In [ ]:
# Print plot for a state monitor
def plot_state_monitor(monitor, neuron):
    plot(monitor.t/ms, monitor.v[neuron]/mV)
    xlabel('Time (ms)')
    ylabel('v');
    
sns.set(style="whitegrid",font_scale=2.5, rc={'figure.figsize':(60,20)})


def seaborn_state_monitor(monitor, neuron): 
    ax = sns.lineplot(monitor.t/ms, monitor.v[neuron]/mV, linewidth=3.0)
    
    '''
    cont = STEP_TIME/ms
    for a in range(0, len(list_coordinates_optimal_path)+1):
        plt.axvline(x=cont*a, color="red")

    ax.xaxis.set_major_locator(ticker.MultipleLocator(25))
    '''
    plt.plot()
    
def seaborn_state_monitor_range(monitor, neuron, init_time, end_time):    
    ax = sns.lineplot(monitor.t[init_time*10:end_time*10]/ms, monitor.v[neuron][init_time*10:end_time*10]/mV, linewidth=4.0)
    
    ax.set_xlabel("Time",fontsize=50, fontweight='bold')
    
    ax.tick_params(labelsize=42)
    ax.tick_params(labelsize=42)
    
    plt.plot()
    
def seaborn_state_monitor_range_comparison(monitor1, monitor2, neuron, init_time, end_time):    
    fig,axs = plt.subplots(nrows=2, ncols=1, sharey='row', sharex='col', figsize=(80,40))
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.01, hspace=0.07)
    
    sns.lineplot(monitor1.t[init_time*10:end_time*10]/ms, monitor1.v[neuron][init_time*10:end_time*10]/mV, linewidth=4.0, ax=axs[0])
    sns.lineplot(monitor2.t[init_time*10:end_time*10]/ms, monitor2.v[neuron][init_time*10:end_time*10]/mV, linewidth=4.0, ax=axs[1])
    
    axs[1].set_xlabel("Time",fontsize=50, fontweight='bold')
    
    axs[0].tick_params(labelsize=42)
    axs[0].set_title("Spontaneous", size=50)
    
    axs[1].tick_params(labelsize=42)
    axs[1].set_title("Attack", size=50)
    
    plt.plot()

In [ ]:
### FUNCTIONS TO LOAD DATA FROM EXTERNAL FILES (TOPOLOGY, WEIGHTS, PARAMETERS...) ###

def getSynapsisDataFromFile(filename):
    synapsysData = []
    minWeight = 0
    maxWeight = 0
    
    with open(filename) as csvfile:
        csvReader = csv.reader(csvfile, delimiter=';')
        next(csvReader, None) # skip header
        
        firstRow =  next(csvReader)
        synapsysData.append([int(firstRow[0]), int(firstRow[1]), float(firstRow[2])])
        minWeight = float(firstRow[2])
        maxWeight = float(firstRow[2])
        
        for row in csvReader:
            synapsysData.append([int(row[0]), int(row[1]), float(row[2])])
            
            if minWeight > float(row[2]):
                minWeight = float(row[2])
            if maxWeight < float(row[2]):
                maxWeight = float(row[2])
    
    return synapsysData, minWeight, maxWeight

def loadIzhikevichParamI(fileName):
    with open(fileName, 'r') as reader:
        I = [line.rstrip('\n') for line in reader]
        
        for i in range(0, len(I)):
            I[i] = float(I[i])*mV/ms
        
        return I
    
def load_initial_voltages(fileName):
    with open(fileName, 'r') as reader:
        v = [line.rstrip('\n') for line in reader]
        
        for i in range(0, len(v)):
            v[i] = float(v[i])*mV
        
        return v


def exportDataStateMonitor(dataStateMon):
    dataStoreStateMon = []
    
    # STATEMON
    # attr (v), time (0-999), neuron (0-199)
    # if execTime == 100ms, len is 1k iterations
    for time in range(0, len(dataStateMon['v'])):
        timeData = []

        for neuron in range(0, len(dataStateMon['v'][time])):
            timeData.append(dataStateMon['v'][time][neuron])
        
        dataStoreStateMon.append(timeData)

     # Result: for each instant, the V of each single neuron
    
    return dataStoreStateMon


def exportDataSpikeMonitor(dataSpikeMon):          
    # SPIKEMON
    # "t": time of each spike during the simulation
    # "i": nb of neuron that spikes, in chronological order. 
    # "count": indicates, for each neuron, the nb of spikes during the simulation

    # Result: array of 2 positions that contains:
    #     0) array where each position has the binome <time instant, nb of neuron>
    #     1) the array "count" with the nb of spikes per neuron
    #
    # Precondition: nb elements in "t" == nb elments in "i"
    
    dataStoreSpikeMon = []
    timeSpikeMon = []
    countSpikeMon = []

    for time in range(0, len(dataSpikeMon['t'])):
        # Each position: <time, nNeuron>
        timeSpikeMon.append([dataSpikeMon['t'][time], dataSpikeMon['i'][time]])
    
    for nSpikes in range(0, len(dataSpikeMon['count'])):
        # Each position: nSpikes
        countSpikeMon.append(dataSpikeMon['count'][nSpikes])
    
    dataStoreSpikeMon = [timeSpikeMon, countSpikeMon]
    
    return dataStoreSpikeMon


def exportAllDataMonitors(stateMons, spikeMons):
    # Generate data to export to CSV (suitable format for later processing)
    dataStateMon1 = exportDataStateMonitor(stateMons[0].get_states(units=False))
    dataStateMon2 = exportDataStateMonitor(stateMons[1].get_states(units=False))
    dataStateMon3 = exportDataStateMonitor(stateMons[2].get_states(units=False))

    dataSpikeMon1 = exportDataSpikeMonitor(spikeMons[0].get_states(units=False))
    dataSpikeMon2 = exportDataSpikeMonitor(spikeMons[1].get_states(units=False))
    dataSpikeMon3 = exportDataSpikeMonitor(spikeMons[2].get_states(units=False))
    
    return [dataStateMon1, dataSpikeMon1, dataStateMon2, dataSpikeMon2, dataStateMon3, dataSpikeMon3]

### FUNCTIONS TO GENERATE RANDOM NUMBER OF NEURONS ###

def generateRandomNeurons(currentNeurons):
    selectedNeurons = []
    
    # Generate random coordinates for the current number of neurons
    for currentRandom in range(0, currentNeurons):

        randNeuron = np.random.randint(0, 200)

        # Avoid duplicates
        while (randNeuron in selectedNeurons):
            randNeuron = np.random.randint(0, 200)

        # Store the selected neuron
        selectedNeurons.append(randNeuron)
        
    return selectedNeurons

    
def generatePairsRandomNeurons(currentNeurons):    
    selectedNeurons = []
    selectedPairsNeurons = []
    
    # Generate random coordinates for the current number of neurons -> pairs of neurons
    for currentRandom in range(0, currentNeurons):
        # Neuron 1 of the pair
        randNeuron1 = np.random.randint(0, 200)

        # Avoid duplicates for neuron 1
        while (randNeuron1 in selectedNeurons):
            randNeuron1 = np.random.randint(0, 200)

        selectedNeurons.append(randNeuron1)

        # Neuron 2 of the pair
        randNeuron2 = np.random.randint(0, 200)

        # Avoid duplicates for neuron 2
        while (randNeuron2 in selectedNeurons):
            randNeuron2 = np.random.randint(0, 200)

        selectedNeurons.append(randNeuron2)

        selectedPairsNeurons.append([randNeuron1, randNeuron2])
        
    return selectedPairsNeurons

### Load simulation parameters from files (weights, synapsis, I param...)

In [ ]:
### VARIABLES FOR THE SIMULATION OF THE ATTACKS ###

BASIC_MODEL = 0
IZHIKEVICH_MODEL = 1

# Load only once synapsis and weights
#dataSynapsisMaze_Conv1, minWeightsMaze_Conv1, maxWeightsMaze_Conv1 = getSynapsisDataFromFile("synapsysMaze-Conv1.csv")
dataSynapsisConv1_Conv2, minWeightsConv1_Conv2, maxWeightsConv1_Conv2 = getSynapsisDataFromFile("synapsysConv1-Conv2.csv")
dataSynapsisConv2_Dense, minWeightsConv2_Dense, maxWeightsConv2_Dense = getSynapsisDataFromFile("synapsysConv2-Dense.csv")

# Process the data for the simulator
initSourceNeuronsConv1_Conv2 = []
initTargetNeuronsConv1_Conv2 = []
initSourceNeuronsConv2_Dense = []
initTargetNeuronsConv2_Dense = []
initWeightsMaze_Conv1 = []
initWeightsConv1_Conv2 = []
initWeightsConv2_Dense = []

for syn in range(0, len(dataSynapsisConv1_Conv2)):
    initSourceNeuronsConv1_Conv2.append(dataSynapsisConv1_Conv2[syn][0])
    initTargetNeuronsConv1_Conv2.append(dataSynapsisConv1_Conv2[syn][1])
    initWeightsConv1_Conv2.append(dataSynapsisConv1_Conv2[syn][2])

for syn in range(0, len(dataSynapsisConv2_Dense)):
    initSourceNeuronsConv2_Dense.append(dataSynapsisConv2_Dense[syn][0])
    initTargetNeuronsConv2_Dense.append(dataSynapsisConv2_Dense[syn][1])
    initWeightsConv2_Dense.append(dataSynapsisConv2_Dense[syn][2])


# Normalize weights for the Izhikevich model
np_initWeightsConv1_Conv2 = np.array(initWeightsConv1_Conv2)
np_initWeightsConv2_Dense = np.array(initWeightsConv2_Dense)

norm_initWeightsConv1_Conv2 = np.interp(np_initWeightsConv1_Conv2, (np_initWeightsConv1_Conv2.min(), np_initWeightsConv1_Conv2.max()), (min_range_V, max_range_V))
norm_initWeightsConv2_Dense = np.interp(np_initWeightsConv2_Dense, (np_initWeightsConv2_Dense.min(), np_initWeightsConv2_Dense.max()), (min_range_V, max_range_V))

N_NEURONS_MAX = 100
N_NEURONS_MIN = 1

attacks_dict = {
    'FLO': ['Flooding', N_NEURONS_MAX, [0.25, 0.5, 0.75, 1.0]], # Stim multiple neurons per t.u.
    'JAM': ['Jamming', N_NEURONS_MAX, [-1.0]], # Inhibit multiple neurons per t.u.
    'SCA': ['PortScanning', N_NEURONS_MIN, [0.25, 0.5, 0.75, 1.0]],  # Stim 1 neuron per t.u.
    'FOR': ['SelectiveForwarding', N_NEURONS_MIN, [-1.0]], # Inhibit 1 neuron per t.u.
    'SPO': ['Spoofing', int(N_NEURONS_MAX/2)], # the attack selects pairs of neurons -> double of this number
    'SYB': ['Sybil', N_NEURONS_MAX],
}

maze =  np.array([
   [ 1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  0.,  0.,  1.,  0.], 
    [ 0.,  0.,  0.,  1.,  1.,  1.,  0.],
    [ 1.,  1.,  1.,  1.,  0.,  0.,  1.],
    [ 1.,  0.,  0.,  0.,  1.,  1.,  1.],
    [ 1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  0.,  1.,  1.,  1.]
])

# Load initial voltages for all simulation (previously generated random values)
v_initial = load_initial_voltages("initial_voltage.txt")

### Normal simulation

In [ ]:
def add_result_simulation(dict_result, neuron, instant):
    if neuron not in dict_result.keys():
        dict_result[neuron] = []

    dict_result[neuron].append(instant)
    
def neuronal_simulation(simulation_duration, params, results):
    #isSequentialAttack, timeStepsSeqAttacks, instantsAttack, stimValue, simulationDuration, neuronList, vIncrement, paramI
    
    start_scope()
    
    #defaultclock.dt = 0.1*ms
    
    # Initial parameters for the equations (typical values)
    # a = 0.02/ms
    # b = 0.2/ms
    # c = -65*mV
    # d = 2*mV/ms
    # I = 0*mV/ms

    # Equations of the Izhikevich neuron model
    eqs = '''
    dv/dt = (0.04/ms/mV)*v**2+(5/ms)*v+140*mV/ms-u + I : volt
    du/dt = a*(b*v-u) : volt/second
    I : volt/second
    a : Hz
    b : Hz
    c : volt
    d : volt/second
    neuronCounter : 1
    positionCounter : 1
    isFirstTime : 1
    timeCounter : second
    '''

    # Reset of Izhikevich model
    reset ='''
    v = c
    u += d
    '''

    thresholdValue = 30
    resetValue = -65

    # Definition of the 1st layer of the CNN
    G = NeuronGroup(276, eqs, threshold='v >= thresholdValue*mV', reset=reset, method='euler')
    # Initialise variables of all neurons (typical values)

    #G.v = resetValue*mV
    G.v = v_initial
    
    G.u = -13*mV/ms # b*v -> 0.2*-65 = -13
    G.a = 0.02/ms
    G.b = 0.2/ms
    G.c = resetValue*mV
    G.d = 8*mV/ms
    #G.I = paramI*mV/ms
    #G.I = loadIzhikevichParamI("paramI.txt")
    G.I = 10*mV/ms
    
    # Create subgroups
    layerConv1 = G[:200]
    layerConv2 = G[200:272]
    layerDense = G[272:276]
    
    # We only store the counters for the first neuron of layer1 (more efficient)   
    layerConv1.neuronCounter[0] = 0
    layerConv1.positionCounter[0] = 0
    layerConv1.timeCounter[0] = 0*ms
    layerConv1.isFirstTime[0] = 1
    
    # SPO uses timeCounter as a list index counter
    if(params["attack"] == "SPO"):
        layerConv1.timeCounter[0] = 0
    
    # Monitors G
    stateMonGlobal = StateMonitor(G, 'v', record=True)
    spikeMonGlobal = SpikeMonitor(G)
    
    # Synapsis definition
    synConv1_Conv2 = Synapses(layerConv1, layerConv2, 'w : volt', on_pre='v_post += w')
    synConv2_Dense = Synapses(layerConv2, layerDense, 'w : volt', on_pre='v_post += w')
    
    # Connect synapsis
    synConv1_Conv2.connect(i=initSourceNeuronsConv1_Conv2, j=initTargetNeuronsConv1_Conv2)
    synConv2_Dense.connect(i=initSourceNeuronsConv2_Dense, j=initTargetNeuronsConv2_Dense)
    
    synConv1_Conv2.w = norm_initWeightsConv1_Conv2*mV
    synConv2_Dense.w = norm_initWeightsConv2_Dense*mV   
    
    # Invoke method only when the rat changes the position
    @network_operation(dt=STEP_TIME)
    def periodicFunction():
        positionCounter = int(layerConv1.positionCounter[0])
        #print("current_counter: ", positionCounter)
        
        if layerConv1.isFirstTime[0] == 1:
            layerConv1.isFirstTime[0] = 0
        else:
            if positionCounter < len(list_coordinates_optimal_path):
                # Reset all neurons to default I value
                G.I = 10*mV/ms
                
                # Get the coordinates of the current position
                coord = list_coordinates_optimal_path[positionCounter]

                # Update I value only for the neurons related to the visible positions from the current one
                list_neurons_index = get_related_neurons_visible_positions(coord[0], coord[1])

                for neuron in list_neurons_index:
                    G.I[neuron] = 15*mV/ms

                # Update the counter of the current position over the maze (for next iteration)
                layerConv1.positionCounter[0] += 1
    
    @network_operation(dt=1*ms)
    def FLO():
        if(params["attack"] == "FLO"):
            current_time_counter = int(layerConv1.timeCounter[0]*1000)
            
            # Allows repetitions of a FLO attack
            current_times_attacks = []
            
            for instant in params["instants_attack"]:
                current_times_attacks.append(int(instant/ms))
            
            #print("------")
            #print("current_time_counter: ", current_time_counter)
            #print("current_times_attacks: ", current_times_attacks)
            
            if(current_time_counter in current_times_attacks):
                for neuron in params["neuron_list"]:
                    #print("neuron: ", neuron)
                    #print("G.v[neuron]: ", G.v[neuron])

                    layerConv1.v[neuron] += params["vIncrement"]*mV
                    
                    add_result_simulation(results, neuron, current_time_counter)

                    #print("NEW G.v[neuron]: ", G.v[neuron])
                    #print("-------------------")
                    
            layerConv1.timeCounter[0] += 1*ms #STEP_TIME
            #print("AFTER layer1.timeCounter[0]: ", layer1.timeCounter[0])
            #print("----------------------------")        
                        
    @network_operation(dt=1*ms)
    def JAM():
        if(params["attack"] == "JAM"):
            current_time_counter = int(layerConv1.timeCounter[0]*1000)
            
            if (current_time_counter >= params["init_attack"]/ms) & (current_time_counter <  params["end_attack"]/ms):
                for neuron in params["neuron_list"]:
                    layerConv1.v[neuron] = -65*mV
                    
                    add_result_simulation(results, neuron, current_time_counter)
            
            layerConv1.timeCounter[0] += 1*ms #STEP_TIME
            
    @network_operation(dt=1*ms)
    def SCA_FOR():        
        if(params["attack"] == "SCA" or params["attack"] == "FOR"):
            neuron = int(layerConv1.neuronCounter[0])
            
            if (layerConv1.timeCounter[0] >= params["init_attack"]):
                if trunc(trunc((layerConv1.timeCounter[0]/ms)) % (params["steps_attack"]/ms)) == 0:
                    if(neuron < 200):
                        
                        #print("neuron: ", neuron)
                        #print("G.v[neuron]: ", G.v[neuron])
                        if params["attack"] == "SCA": 
                            layerConv1.v[neuron] += params["vIncrement"]*mV
                            add_result_simulation(results, neuron, int(layerConv1.timeCounter[0]*1000))
                        else: #FOR
                            if (layerConv1.v[neuron] <= -65*mV): # Keep normal behavior
                                pass
                            elif (layerConv1.v[neuron] - params["vIncrement"]*mV) >= -65*mV:
                                layerConv1.v[neuron] -= params["vIncrement"]*mV
                                add_result_simulation(results, neuron, int(layerConv1.timeCounter[0]*1000))
                            else:
                                layerConv1.v[neuron] = -65*mV
                                add_result_simulation(results, neuron, int(layerConv1.timeCounter[0]*1000))
                        #print("NEW G.v[neuron]: ", G.v[neuron])
                        #print("-------------------")  
                        
                        
                            
                        layerConv1.neuronCounter[0] += 1
                        
            layerConv1.timeCounter[0] += 1*ms
    
    @network_operation(dt=0.1*ms)
    def SPO():
        if(params["attack"] == "SPO"):
            current_time_counter = round(layerConv1.timeCounter[0]/ms, 2)
            #print("simulation instant: ", current_time_counter)
            
            if (current_time_counter >= params["init_attack"]/ms) & (current_time_counter <  params["end_attack"]/ms):
                # Counter to determine the index within the lists to access the attack voltages
                current_counter = int(layerConv1.neuronCounter[0])
                #print("attack instant: ", current_counter)
                
                # Change the voltage of the targeted neurons according to current_counter
                for neuron in params["neuron_list_attack"]:
                    #print("neuron: ", neuron)
                    #print("G.v[neuron]: ", G.v[neuron])
                    layerConv1.v[neuron] = params["recorded_activity"][neuron][current_counter]
                    #print("NEW G.v[neuron]: ", G.v[neuron])
                    #print("-------------------")
                
                layerConv1.neuronCounter[0] += 1
            
            layerConv1.timeCounter[0] += 0.1*ms #STEP_TIME
        
    @network_operation(dt=1*ms)
    def SYB():
        if(params["attack"] == "SYB"):
            current_time_counter = int(layerConv1.timeCounter[0]*1000)
            
            # Allows repetitions of a SYB attack
            current_times_attacks = []
            
            for instant in params["instants_attack"]:
                current_times_attacks.append(int(instant/ms))
            
            if(current_time_counter in current_times_attacks):
                #print("instant: ", current_time_counter)
                for neuron in params["neuron_list"]:
                    #print("nNeuron: ", neuron)
                    #print("layerConv1.v[neuron]: ", layerConv1.v[neuron])
                    
                    old_voltage = layerConv1.v[neuron]
                    layerConv1.v[neuron] = resetValue*mV + thresholdValue*mV - layerConv1.v[neuron]
                    #print("NEW layerConv1.v[neuron]: ", layerConv1.v[neuron])
                    
                    if layerConv1.v[neuron] < old_voltage:
                        add_result_simulation(results["inhibition"], neuron, current_time_counter)
                    else:
                        add_result_simulation(results["stimulation"], neuron, current_time_counter)
                    
            layerConv1.timeCounter[0] += 1*ms #STEP_TIME
        
    
    @network_operation(dt=1*ms)
    def SIN():
        if(params["attack"] == "SIN"):
            current_time_counter = int(layerConv1.timeCounter[0]*1000)
            current_times_attacks = []
            
            for instant in params["instants_attack"]:
                current_times_attacks.append(int(instant/ms))
            
            if(current_time_counter in current_times_attacks):
                for neuron in params["neuron_list"]:
                    layerConv1.v[neuron] += params["vIncrement"]*mV
                    
                    add_result_simulation(results, neuron, current_time_counter)
                    
            layerConv1.timeCounter[0] += 1*ms #STEP_TIME
            
            
    @network_operation(dt=1*ms)
    def NON():
        if(params["attack"] == "NON"):           
            if (layerConv1.timeCounter[0] >= params["init_attack"]):
                if trunc(trunc(layerConv1.timeCounter[0]/ms - params["init_attack"]/ms) % (params["steps_attack"]/ms)) == 0:
                    #print("instant attack: ",layerConv1.timeCounter[0])
                    for neuron in params["neuron_list"]:
                        action = random.choices([1,2,3], weights=(params["probability_spontaneous"], 
                                                               params["probability_stimulation"], 
                                                               params["probability_inhibition"]), k=1)[0]
                        
                        #print("neuron: %s, action: %s" %(neuron, action))
                        if action == 2: # Stimulate
                            #print("layerConv1.v[neuron]: ", layerConv1.v[neuron])
                            layerConv1.v[neuron] += params["vIncrement"]*mV
                            add_result_simulation(results["stimulation"], neuron, int(layerConv1.timeCounter[0]*1000))
                            #print("NEW G.v[neuron]: ", G.v[neuron])
                            #print("-------------------")
                        elif action == 3: # Inhibit
                            #print("layerConv1.v[neuron]: ", layerConv1.v[neuron])
                            if (layerConv1.v[neuron] <= -65*mV): # Keep normal behavior
                                pass
                            elif (layerConv1.v[neuron] - params["vIncrement"]*mV) >= -65*mV:
                                layerConv1.v[neuron] -= params["vIncrement"]*mV
                                add_result_simulation(results["inhibition"], neuron, int(layerConv1.timeCounter[0]*1000))
                            else:
                                layerConv1.v[neuron] = -65*mV
                                add_result_simulation(results["inhibition"], neuron, int(layerConv1.timeCounter[0]*1000))
                            #print("NEW G.v[neuron]: ", G.v[neuron])
                            #print("-------------------")
                        
            layerConv1.timeCounter[0] += 1*ms
            
    run(simulation_duration)
    
    return [spikeMonGlobal, stateMonGlobal]

### Tests for comparating between different combination of parameters

In [ ]:
csv_header_neurons_global = [
    "test", "n_attacks", "n_neurons", "stim_value", "vIncrement", "paramI",
    "glbl_i_spikes_mean","glbl_a_spikes_mean","glbl_i_spikes_std","glbl_a_spikes_std","glbl_shifts_mean",
    "attk_i_spikes_mean","attk_a_spikes_mean","attk_i_spikes_std","attk_a_spikes_std","attk_shifts_mean",
]

csv_header_neurons_layers = [
    "test", "n_attacks", "n_neurons", "stim_value", "vIncrement", "paramI", "t_window",
    "l1_glbl_i_spikes_mean","l1_glbl_a_spikes_mean","l1_glbl_i_spikes_std","l1_glbl_a_spikes_std","l1_glbl_shifts_mean",
    "l1_attk_i_spikes_mean","l1_attk_a_spikes_mean","l1_attk_i_spikes_std","l1_attk_a_spikes_std","l1_attk_shifts_mean",
    "l2_glbl_i_spikes_mean","l2_glbl_a_spikes_mean","l2_glbl_i_spikes_std","l2_glbl_a_spikes_std","l2_glbl_shifts_mean",
    "l2_attk_i_spikes_mean","l2_attk_a_spikes_mean","l2_attk_i_spikes_std","l2_attk_a_spikes_std","l2_attk_shifts_mean",
    "l3_glbl_i_spikes_mean","l3_glbl_a_spikes_mean","l3_glbl_i_spikes_std","l3_glbl_a_spikes_std","l3_glbl_shifts_mean",
    "l3_attk_i_spikes_mean","l3_attk_a_spikes_mean","l3_attk_i_spikes_std","l3_attk_a_spikes_std","l3_attk_shifts_mean",
]

# Stats files
stats_dir_FLO = stats_dir+"FLO/"
stats_dir_JAM = stats_dir+"JAM/"
stats_dir_SCA = stats_dir+"SCA/"
stats_dir_FOR = stats_dir+"FOR/"
stats_dir_SPO = stats_dir+"SPO/"
stats_dir_SYB = stats_dir+"SYB/"
stats_dir_SIN = stats_dir+"SIN/"
stats_dir_NON = stats_dir+"NON/"
    
csv_filename_FLO = stats_dir_FLO+"temp_export_FLO.csv"
csv_filename_JAM = stats_dir_JAM+"temp_export_JAM.csv"
csv_filename_SCA = stats_dir_SCA+"temp_export_SCA.csv"
csv_filename_FOR = stats_dir_FOR+"temp_export_FOR.csv"
csv_filename_SPO = stats_dir_SPO+"temp_export_SPO.csv"
csv_filename_SYB = stats_dir_SYB+"temp_export_SYB.csv"
csv_filename_SIN = stats_dir_SIN+"temp_export_SIN.csv"
csv_filename_NON = stats_dir_NON+"temp_export_NON.csv"

stats_neurons_global_csv_filename_FLO = stats_dir_FLO+"stats_neurons_global_FLO.csv"
stats_neurons_layers_csv_filename_FLO = stats_dir_FLO+"stats_neurons_layers_FLO.csv"

stats_neurons_global_csv_filename_JAM = stats_dir_JAM+"stats_neurons_global_JAM.csv"
stats_neurons_layers_csv_filename_JAM = stats_dir_JAM+"stats_neurons_layers_JAM.csv"

stats_neurons_global_csv_filename_SCA = stats_dir_SCA+"stats_neurons_global_SCA.csv"
stats_neurons_layers_csv_filename_SCA = stats_dir_SCA+"stats_neurons_layers_SCA.csv"

stats_neurons_global_csv_filename_FOR = stats_dir_FOR+"stats_neurons_global_FOR.csv"
stats_neurons_layers_csv_filename_FOR = stats_dir_FOR+"stats_neurons_layers_FOR.csv"

stats_neurons_global_csv_filename_SPO = stats_dir_SPO+"stats_neurons_global_SPO.csv"
stats_neurons_layers_csv_filename_SPO = stats_dir_SPO+"stats_neurons_layers_SPO.csv"

stats_neurons_global_csv_filename_SYB = stats_dir_SYB+"stats_neurons_global_SYB.csv"
stats_neurons_layers_csv_filename_SYB = stats_dir_SYB+"stats_neurons_layers_SYB.csv"

stats_neurons_global_csv_filename_SIN = stats_dir_SIN+"stats_neurons_global_SIN.csv"
stats_neurons_layers_csv_filename_SIN = stats_dir_SIN+"stats_neurons_layers_SIN.csv"

stats_neurons_global_csv_filename_NON = stats_dir_NON+"stats_neurons_global_NON.csv"
stats_neurons_layers_csv_filename_NON = stats_dir_NON+"stats_neurons_layers_NON.csv"

stats_dispersion_csv_filename_FLO = stats_dir_FLO+"stats_dispersion_FLO.csv"
stats_dispersion_csv_filename_JAM = stats_dir_JAM+"stats_dispersion_JAM.csv"
stats_dispersion_csv_filename_SCA = stats_dir_SCA+"stats_dispersion_SCA.csv"
stats_dispersion_csv_filename_FOR = stats_dir_FOR+"stats_dispersion_FOR.csv"
stats_dispersion_csv_filename_SPO = stats_dir_SPO+"stats_dispersion_SPO.csv"
stats_dispersion_csv_filename_SYB = stats_dir_SYB+"stats_dispersion_SYB.csv"
stats_dispersion_csv_filename_SIN = stats_dir_SIN+"stats_dispersion_SIN.csv"
stats_dispersion_csv_filename_NON = stats_dir_NON+"stats_dispersion_NON.csv"

output_file_FLO = stats_dir_FLO+"output_moving_FLO.txt"
output_file_JAM = stats_dir_JAM+"output_moving_JAM.txt"
output_file_SCA = stats_dir_SCA+"output_moving_SCA.txt"
output_file_FOR = stats_dir_FOR+"output_moving_FOR.txt"
output_file_SPO = stats_dir_SPO+"output_moving_SPO.txt"
output_file_SYB = stats_dir_SYB+"output_moving_SYB.txt"
output_file_SIN = stats_dir_SIN+"output_moving_SIN.txt"
output_file_NON = stats_dir_NON+"output_moving_NON.txt"

# Plots
plots_dir_FLO_raster = plots_dir+"FLO/rasters/"
plots_dir_FLO_spikes = plots_dir+"FLO/spikes/"
plots_dir_FLO_dispersion = plots_dir+"FLO/dispersion/"
plots_dir_FLO_shifts = plots_dir+"FLO/shifts/"

plots_dir_JAM_raster = plots_dir+"JAM/rasters/"
plots_dir_JAM_spikes = plots_dir+"JAM/spikes/"
plots_dir_JAM_dispersion = plots_dir+"JAM/dispersion/"
plots_dir_JAM_shifts = plots_dir+"JAM/shifts/"

plots_dir_SCA_raster = plots_dir+"SCA/rasters/"
plots_dir_SCA_spikes = plots_dir+"SCA/spikes/"
plots_dir_SCA_dispersion = plots_dir+"SCA/dispersion/"
plots_dir_SCA_shifts = plots_dir+"SCA/shifts/"

plots_dir_FOR_raster = plots_dir+"FOR/rasters/"
plots_dir_FOR_spikes = plots_dir+"FOR/spikes/"
plots_dir_FOR_dispersion = plots_dir+"FOR/dispersion/"
plots_dir_FOR_shifts = plots_dir+"FOR/shifts/"

plots_dir_SPO_raster = plots_dir+"SPO/rasters/"
plots_dir_SPO_spikes = plots_dir+"SPO/spikes/"
plots_dir_SPO_dispersion = plots_dir+"SPO/dispersion/"
plots_dir_SPO_shifts = plots_dir+"SPO/shifts/"

plots_dir_SYB_raster = plots_dir+"SYB/rasters/"
plots_dir_SYB_spikes = plots_dir+"SYB/spikes/"
plots_dir_SYB_dispersion = plots_dir+"SYB/dispersion/"
plots_dir_SYB_shifts = plots_dir+"SYB/shifts/"

plots_dir_SIN_raster = plots_dir+"SIN/rasters/"
plots_dir_SIN_spikes = plots_dir+"SIN/spikes/"
plots_dir_SIN_dispersion = plots_dir+"SIN/dispersion/"
plots_dir_SIN_shifts = plots_dir+"SIN/shifts/"

plots_dir_NON_raster = plots_dir+"NON/rasters/"
plots_dir_NON_spikes = plots_dir+"NON/spikes/"
plots_dir_NON_dispersion = plots_dir+"NON/dispersion/"
plots_dir_NON_shifts = plots_dir+"NON/shifts/"

# Raster plots
csv_rasters_FLO = stats_dir_FLO+"rasters_FLO.csv"
csv_rasters_JAM = stats_dir_JAM+"rasters_JAM.csv"
csv_rasters_SCA = stats_dir_SCA+"rasters_SCA.csv"
csv_rasters_FOR = stats_dir_FOR+"rasters_FOR.csv"
csv_rasters_SPO = stats_dir_SPO+"rasters_SPO.csv"
csv_rasters_SYB = stats_dir_SYB+"rasters_SYB.csv"
csv_rasters_SIN = stats_dir_SIN+"rasters_SIN.csv"
csv_rasters_NON = stats_dir_NON+"rasters_NON.csv"

# Raster aggregated ("cloud of dots") for each position
aggr_csv_filename_FLO = stats_dir_FLO+"aggr_FLO.csv"
aggr_csv_filename_JAM = stats_dir_JAM+"aggr_JAM.csv"
aggr_csv_filename_SCA = stats_dir_SCA+"aggr_SCA.csv"
aggr_csv_filename_FOR = stats_dir_FOR+"aggr_FOR.csv"
aggr_csv_filename_SPO = stats_dir_SPO+"aggr_SPO.csv"
aggr_csv_filename_SYB = stats_dir_SYB+"aggr_SYB.csv"
aggr_csv_filename_SIN = stats_dir_SIN+"aggr_SIN.csv"
aggr_csv_filename_NON = stats_dir_NON+"aggr_NON.csv"

condition_generate_rasters = {
    "n_neurons": [100],
    "v_increment": [40],
}

T1_init = 0
T1_fin = 1000

T2_init = 12000
T2_fin = 13000

T3_init = 26000
T3_fin = 27000

In [ ]:
# Create directories if they do not exist

if "FLO" in list_attack_generation:    
    # Stats
    if not os.path.exists(stats_dir_FLO):
        os.makedirs(stats_dir_FLO)
        
    # Plots
    if not os.path.exists(plots_dir_FLO_raster):
        os.makedirs(plots_dir_FLO_raster)

    if not os.path.exists(plots_dir_FLO_spikes):
        os.makedirs(plots_dir_FLO_spikes)

    if not os.path.exists(plots_dir_FLO_dispersion):
        os.makedirs(plots_dir_FLO_dispersion)

    if not os.path.exists(plots_dir_FLO_shifts):
        os.makedirs(plots_dir_FLO_shifts)

if "JAM" in list_attack_generation:    
    # Stats
    if not os.path.exists(stats_dir_JAM):
        os.makedirs(stats_dir_JAM)
        
    # Plots
    if not os.path.exists(plots_dir_JAM_raster):
        os.makedirs(plots_dir_JAM_raster)

    if not os.path.exists(plots_dir_JAM_spikes):
        os.makedirs(plots_dir_JAM_spikes)

    if not os.path.exists(plots_dir_JAM_dispersion):
        os.makedirs(plots_dir_JAM_dispersion)

    if not os.path.exists(plots_dir_JAM_shifts):
        os.makedirs(plots_dir_JAM_shifts)

if "SCA" in list_attack_generation:
    # Stats
    if not os.path.exists(stats_dir_SCA):
        os.makedirs(stats_dir_SCA)

    # Plots
    if not os.path.exists(plots_dir_SCA_raster):
        os.makedirs(plots_dir_SCA_raster)

    if not os.path.exists(plots_dir_SCA_spikes):
        os.makedirs(plots_dir_SCA_spikes)

    if not os.path.exists(plots_dir_SCA_dispersion):
        os.makedirs(plots_dir_SCA_dispersion)

    if not os.path.exists(plots_dir_SCA_shifts):
        os.makedirs(plots_dir_SCA_shifts)
        
if "FOR" in list_attack_generation:    
    # Stats
    if not os.path.exists(stats_dir_FOR):
        os.makedirs(stats_dir_FOR)
        
    # Plots
    if not os.path.exists(plots_dir_FOR_raster):
        os.makedirs(plots_dir_FOR_raster)

    if not os.path.exists(plots_dir_FOR_spikes):
        os.makedirs(plots_dir_FOR_spikes)

    if not os.path.exists(plots_dir_FOR_dispersion):
        os.makedirs(plots_dir_FOR_dispersion)

    if not os.path.exists(plots_dir_FOR_shifts):
        os.makedirs(plots_dir_FOR_shifts)
        
if "SPO" in list_attack_generation:    
    # Stats
    if not os.path.exists(stats_dir_SPO):
        os.makedirs(stats_dir_SPO)
        
    # Plots
    if not os.path.exists(plots_dir_SPO_raster):
        os.makedirs(plots_dir_SPO_raster)

    if not os.path.exists(plots_dir_SPO_spikes):
        os.makedirs(plots_dir_SPO_spikes)

    if not os.path.exists(plots_dir_SPO_dispersion):
        os.makedirs(plots_dir_SPO_dispersion)

    if not os.path.exists(plots_dir_SPO_shifts):
        os.makedirs(plots_dir_SPO_shifts)
        
if "SYB" in list_attack_generation:    
    # Stats
    if not os.path.exists(stats_dir_SYB):
        os.makedirs(stats_dir_SYB)
        
    # Plots
    if not os.path.exists(plots_dir_SYB_raster):
        os.makedirs(plots_dir_SYB_raster)

    if not os.path.exists(plots_dir_SYB_spikes):
        os.makedirs(plots_dir_SYB_spikes)

    if not os.path.exists(plots_dir_SYB_dispersion):
        os.makedirs(plots_dir_SYB_dispersion)

    if not os.path.exists(plots_dir_SYB_shifts):
        os.makedirs(plots_dir_SYB_shifts)
        
if "SIN" in list_attack_generation:    
    # Stats
    if not os.path.exists(stats_dir_SIN):
        os.makedirs(stats_dir_SIN)
        
    # Plots
    if not os.path.exists(plots_dir_SIN_raster):
        os.makedirs(plots_dir_SIN_raster)

    if not os.path.exists(plots_dir_SIN_spikes):
        os.makedirs(plots_dir_SIN_spikes)

    if not os.path.exists(plots_dir_SIN_dispersion):
        os.makedirs(plots_dir_SIN_dispersion)

    if not os.path.exists(plots_dir_SIN_shifts):
        os.makedirs(plots_dir_SIN_shifts)
        
if "NON" in list_attack_generation:    
    # Stats
    if not os.path.exists(stats_dir_NON):
        os.makedirs(stats_dir_NON)
        
    # Plots
    if not os.path.exists(plots_dir_NON_raster):
        os.makedirs(plots_dir_NON_raster)

    if not os.path.exists(plots_dir_NON_spikes):
        os.makedirs(plots_dir_NON_spikes)

    if not os.path.exists(plots_dir_NON_dispersion):
        os.makedirs(plots_dir_NON_dispersion)

    if not os.path.exists(plots_dir_NON_shifts):
        os.makedirs(plots_dir_NON_shifts)

In [ ]:
if "FLO" in list_attack_generation:
    open(csv_filename_FLO, 'w').close() # Remove file content
    append_to_csv_file(csv_filename_FLO, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster files
    open(csv_rasters_FLO, 'w').close() # Remove file content
    append_to_csv_file(csv_rasters_FLO, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster aggregated files
    open(aggr_csv_filename_FLO, 'w').close() # Remove file content
    append_to_csv_file(aggr_csv_filename_FLO, ["time_delta", "number_spikes", "position_attack", "attack", "position", "n_neurons", "v_increment", "n_exec"])

    # Overwrite stats neurons CSV
    open(stats_neurons_global_csv_filename_FLO, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_global_csv_filename_FLO, csv_header_neurons_global)

    open(stats_neurons_layers_csv_filename_FLO, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_layers_csv_filename_FLO, csv_header_neurons_layers)
    
    # Output file
    open(output_file_FLO, 'w').close() # Remove file content
    
if "SCA" in list_attack_generation:
    open(csv_filename_SCA, 'w').close() # Remove file content
    append_to_csv_file(csv_filename_SCA, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])

    # Overwrite raster files
    open(csv_rasters_SCA, 'w').close() # Remove file content
    append_to_csv_file(csv_rasters_SCA, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])

    # Overwrite raster aggregated files
    open(aggr_csv_filename_SCA, 'w').close() # Remove file content
    append_to_csv_file(aggr_csv_filename_SCA, ["time_delta", "number_spikes", "position_attack", "attack", "position", "n_neurons", "v_increment", "n_exec"])
    
    # Overwrite stats neurons CSV
    open(stats_neurons_global_csv_filename_SCA, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_global_csv_filename_SCA, csv_header_neurons_global)

    open(stats_neurons_layers_csv_filename_SCA, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_layers_csv_filename_SCA, csv_header_neurons_layers)
    
    # Output file
    open(output_file_SCA, 'w').close() # Remove file content

if "JAM" in list_attack_generation:
    open(csv_filename_JAM, 'w').close() # Remove file content
    append_to_csv_file(csv_filename_JAM, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster files
    open(csv_rasters_JAM, 'w').close() # Remove file content
    append_to_csv_file(csv_rasters_JAM, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster aggregated files
    open(aggr_csv_filename_JAM, 'w').close() # Remove file content
    append_to_csv_file(aggr_csv_filename_JAM, ["time_delta", "number_spikes", "position_attack", "attack", "position", "n_neurons", "v_increment", "n_exec"])

    # Overwrite stats neurons CSV
    open(stats_neurons_global_csv_filename_JAM, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_global_csv_filename_JAM, csv_header_neurons_global)

    open(stats_neurons_layers_csv_filename_JAM, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_layers_csv_filename_JAM, csv_header_neurons_layers)
    
    # Output file
    open(output_file_JAM, 'w').close() # Remove file content
    
if "FOR" in list_attack_generation:
    open(csv_filename_FOR, 'w').close() # Remove file content
    append_to_csv_file(csv_filename_FOR, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster files
    open(csv_rasters_FOR, 'w').close() # Remove file content
    append_to_csv_file(csv_rasters_FOR, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster aggregated files
    open(aggr_csv_filename_FOR, 'w').close() # Remove file content
    append_to_csv_file(aggr_csv_filename_FOR, ["time_delta", "number_spikes", "position_attack", "attack", "position", "n_neurons", "v_increment", "n_exec"])

    # Overwrite stats neurons CSV
    open(stats_neurons_global_csv_filename_FOR, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_global_csv_filename_FOR, csv_header_neurons_global)

    open(stats_neurons_layers_csv_filename_FOR, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_layers_csv_filename_FOR, csv_header_neurons_layers)
    
    # Output file
    open(output_file_FOR, 'w').close() # Remove file content
    
if "SPO" in list_attack_generation:
    open(csv_filename_SPO, 'w').close() # Remove file content
    append_to_csv_file(csv_filename_SPO, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster files
    open(csv_rasters_SPO, 'w').close() # Remove file content
    append_to_csv_file(csv_rasters_SPO, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster aggregated files
    open(aggr_csv_filename_SPO, 'w').close() # Remove file content
    append_to_csv_file(aggr_csv_filename_SPO, ["time_delta", "number_spikes", "position_attack", "attack", "position", "n_neurons", "v_increment", "n_exec"])

    # Overwrite stats neurons CSV
    open(stats_neurons_global_csv_filename_SPO, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_global_csv_filename_SPO, csv_header_neurons_global)

    open(stats_neurons_layers_csv_filename_SPO, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_layers_csv_filename_SPO, csv_header_neurons_layers)
    
    # Output file
    open(output_file_SPO, 'w').close() # Remove file content
    
if "SYB" in list_attack_generation:
    open(csv_filename_SYB, 'w').close() # Remove file content
    append_to_csv_file(csv_filename_SYB, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster files
    open(csv_rasters_SYB, 'w').close() # Remove file content
    append_to_csv_file(csv_rasters_SYB, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster aggregated files
    open(aggr_csv_filename_SYB, 'w').close() # Remove file content
    append_to_csv_file(aggr_csv_filename_SYB, ["time_delta", "number_spikes", "position_attack", "attack", "position", "n_neurons", "v_increment", "n_exec"])

    # Overwrite stats neurons CSV
    open(stats_neurons_global_csv_filename_SYB, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_global_csv_filename_SYB, csv_header_neurons_global)

    open(stats_neurons_layers_csv_filename_SYB, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_layers_csv_filename_SYB, csv_header_neurons_layers)
    
    # Output file
    open(output_file_SYB, 'w').close() # Remove file content
    
if "SIN" in list_attack_generation:
    open(csv_filename_SIN, 'w').close() # Remove file content
    append_to_csv_file(csv_filename_SIN, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster files
    open(csv_rasters_SIN, 'w').close() # Remove file content
    append_to_csv_file(csv_rasters_SIN, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster aggregated files
    open(aggr_csv_filename_SIN, 'w').close() # Remove file content
    append_to_csv_file(aggr_csv_filename_SIN, ["time_delta", "number_spikes", "position_attack", "attack", "position", "n_neurons", "v_increment", "n_exec"])

    # Overwrite stats neurons CSV
    open(stats_neurons_global_csv_filename_SIN, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_global_csv_filename_SIN, csv_header_neurons_global)

    open(stats_neurons_layers_csv_filename_SIN, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_layers_csv_filename_SIN, csv_header_neurons_layers)
    
    # Output file
    open(output_file_SIN, 'w').close() # Remove file content
    
if "NON" in list_attack_generation:
    open(csv_filename_NON, 'w').close() # Remove file content
    append_to_csv_file(csv_filename_NON, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster files
    open(csv_rasters_NON, 'w').close() # Remove file content
    append_to_csv_file(csv_rasters_NON, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster aggregated files
    open(aggr_csv_filename_NON, 'w').close() # Remove file content
    append_to_csv_file(aggr_csv_filename_NON, ["time_delta", "number_spikes", "position_attack", "attack", "position", "n_neurons", "v_increment", "n_exec"])

    # Overwrite stats neurons CSV
    open(stats_neurons_global_csv_filename_NON, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_global_csv_filename_NON, csv_header_neurons_global)

    open(stats_neurons_layers_csv_filename_NON, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_layers_csv_filename_NON, csv_header_neurons_layers)
    
    # Output file
    open(output_file_NON, 'w').close() # Remove file content

In [ ]:
def generate_stats_neurons_pandas(test_id, position_attack, attack, csv_filename, stats_neurons_global_csv_filename, stats_neurons_layers_csv_filename):
    df = pd.read_csv(csv_filename, delimiter=";", dtype={'n_neurons': np.int32})
    
    #### 1) Aggregated statistics for all layers (global)
    data_initial_state = df[df.attack == "initial_state"]
    df_attack = df[df.attack == attack]
    
    list_attacked_neurons = df_attack["attacked_neurons"].unique()[0].split("-")
    
    grouped = df_attack.groupby('neuron').count()["time_delta"]
    d_attack_aggr_neuron = pd.DataFrame()
    d_attack_aggr_neuron['neuron'] = grouped.index
    d_attack_aggr_neuron['n_spikes'] = grouped.tolist()
    
    # Get the data from the initial_state experiment based on the paramI value
    param_I = df_attack.paramI.unique()[0]
    
    grouped = data_initial_state[(data_initial_state.paramI == param_I)].groupby('neuron').count()["time_delta"]
    d_initial_aggr_neuron = pd.DataFrame()
    d_initial_aggr_neuron['neuron'] = grouped.index
    d_initial_aggr_neuron['n_spikes'] = grouped.tolist()
    
    data_csv = []
    data_csv.append(test_id)
    data_csv.append(position_attack)
    data_csv.append(df_attack["n_attacks"].unique()[0])
    data_csv.append(df_attack["n_neurons"].unique()[0])
    data_csv.append(df_attack["stim_value"].unique()[0])
    data_csv.append(df_attack["vIncrement"].unique()[0])
    data_csv.append(df_attack["paramI"].unique()[0])   
    
    ### GLOBAL PERSPECTIVE
    # Total number of spikes
    initial_nSpikes = data_initial_state[(data_initial_state.paramI == param_I)].count()["neuron"]
    attack_nSpikes = df_attack.count()["neuron"]

    # Mean of spikes (over the total neurons of the layer)
    initial_mean_spikes = round(d_initial_aggr_neuron["n_spikes"].mean(), 2)
    attack_mean_spikes = round(d_attack_aggr_neuron["n_spikes"].mean(), 2)

    # Standard deviation (over the total neurons of the layer)
    initial_std_spikes = round(d_initial_aggr_neuron["n_spikes"].std(), 2)
    attack_std_spikes = round(d_attack_aggr_neuron["n_spikes"].std(), 2)

    # Percentage of shifts between the attack and normal behaviour
    initial_nSpikes_join = data_initial_state[(data_initial_state.paramI == param_I)][["attack", "time_delta", "neuron"]].copy()
    attack_nSpikes_join = df_attack[["attack", "time_delta", "neuron"]].copy()
    join_initial_attack_shifts = pd.merge(attack_nSpikes_join, initial_nSpikes_join,  how='left', left_on=['time_delta','neuron'], right_on = ['time_delta','neuron'])
    join_initial_attack_shifts = join_initial_attack_shifts.fillna(0)
    n_shifts_initial_attack = join_initial_attack_shifts[join_initial_attack_shifts.attack_y == 0].count()["neuron"]
    percent_shifts_initial_attack = round((n_shifts_initial_attack/attack_nSpikes)*100,2)

    # List of neurons shifted in this layer
    attack_shifted_neurons = join_initial_attack_shifts[join_initial_attack_shifts.attack_y == 0]["neuron"].unique()

    ### ATTACKED NEURONS PERSPECTIVE

    # spikes: mean
    a_initial_mean_spikes = round(d_initial_aggr_neuron[(d_initial_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].mean(), 2)
    a_attack_mean_spikes = round(d_attack_aggr_neuron[(d_attack_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].mean(), 2)

    # Standard deviation
    a_initial_std_spikes = round(d_initial_aggr_neuron[(d_initial_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].std(), 2)
    a_attack_std_spikes = round(d_attack_aggr_neuron[(d_attack_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].std(), 2)

    # mean of % shifts
    a_join_shifts_attk = join_initial_attack_shifts[join_initial_attack_shifts.neuron.isin(attack_shifted_neurons)]
    a_n_shifts_zeros = a_join_shifts_attk[a_join_shifts_attk.attack_y == 0].count()["neuron"]
    # number of shifts under attack (count zeros) / total number of spikes under attack 
    a_mean_shifts = round((a_n_shifts_zeros/a_join_shifts_attk.neuron.count())*100, 2)

    data_csv.append(initial_mean_spikes)
    data_csv.append(attack_mean_spikes)
    data_csv.append(initial_std_spikes)
    data_csv.append(attack_std_spikes)
    data_csv.append(percent_shifts_initial_attack)

    data_csv.append(a_initial_mean_spikes)
    data_csv.append(a_attack_mean_spikes)
    data_csv.append(a_initial_std_spikes)
    data_csv.append(a_attack_std_spikes)
    data_csv.append(a_mean_shifts)
    
    append_to_csv_file(stats_neurons_global_csv_filename, data_csv)
    
    #### 2) Statistics per each layer ####
    for position in range(1, 4):
        if position == 1:
            min_range_time = T1_init
            max_range_time = T1_fin
        elif position == 2:
            min_range_time = T2_init
            max_range_time = T2_fin
        elif position == 3:
            min_range_time = T3_init
            max_range_time = T3_fin
        
        data_csv = []
        data_csv.append(test_id)
        data_csv.append(position_attack)
        data_csv.append(df_attack["n_attacks"].unique()[0])
        data_csv.append(df_attack["n_neurons"].unique()[0])
        data_csv.append(df_attack["stim_value"].unique()[0])
        data_csv.append(df_attack["vIncrement"].unique()[0])
        data_csv.append(df_attack["paramI"].unique()[0])
        data_csv.append(position)
        
        # We need to filter by time_delta before calculating statistics        
        data_initial_state = df[(df.attack == "initial_state") & (df.time_delta >= min_range_time) & (df.time_delta < max_range_time)]
        df_attack = df[(df.attack == attack) & (df.time_delta >= min_range_time) & (df.time_delta < max_range_time)]

        list_attacked_neurons = df_attack["attacked_neurons"].unique()[0].split("-")

        grouped = df_attack.groupby('neuron').count()["time_delta"]
        d_attack_aggr_neuron = pd.DataFrame()
        d_attack_aggr_neuron['neuron'] = grouped.index
        d_attack_aggr_neuron['n_spikes'] = grouped.tolist()

        # Get the data from the initial_state experiment based on the paramI value
        param_I = df_attack.paramI.unique()[0]

        grouped = data_initial_state[(data_initial_state.paramI == param_I)].groupby('neuron').count()["time_delta"]
        d_initial_aggr_neuron = pd.DataFrame()
        d_initial_aggr_neuron['neuron'] = grouped.index
        d_initial_aggr_neuron['n_spikes'] = grouped.tolist()
        
        for layer in range(1, 4):
            if layer == 1:
                min_range_neuron = 0
                max_range_neuron = 200
            elif layer == 2:
                min_range_neuron = 200
                max_range_neuron = 272
            elif layer == 3:
                min_range_neuron = 272
                max_range_neuron = 276

            ### GLOBAL PERSPECTIVE
            # Total number of spikes
            initial_nSpikes = data_initial_state[(data_initial_state.paramI == param_I) & (data_initial_state.neuron >= min_range_neuron) & (data_initial_state.neuron < max_range_neuron)].count()["neuron"]
            attack_nSpikes = df_attack[(df_attack.neuron >= min_range_neuron) & (df_attack.neuron < max_range_neuron)].count()["neuron"]

            # Mean of spikes (over the total neurons of the layer)
            initial_mean_spikes = round(d_initial_aggr_neuron[(d_initial_aggr_neuron.neuron >= min_range_neuron) & (d_initial_aggr_neuron.neuron < max_range_neuron)]["n_spikes"].mean(), 2)
            attack_mean_spikes = round(d_attack_aggr_neuron[(d_attack_aggr_neuron.neuron >= min_range_neuron) & (d_attack_aggr_neuron.neuron < max_range_neuron)]["n_spikes"].mean(), 2)

            # Standard deviation (over the total neurons of the layer)
            initial_std_spikes = round(d_initial_aggr_neuron[(d_initial_aggr_neuron.neuron >= min_range_neuron) & (d_initial_aggr_neuron.neuron < max_range_neuron)]["n_spikes"].std(), 2)
            attack_std_spikes = round(d_attack_aggr_neuron[(d_attack_aggr_neuron.neuron >= min_range_neuron) & (d_attack_aggr_neuron.neuron < max_range_neuron)]["n_spikes"].std(), 2)

            # Percentage of shifts between the attack and normal behaviour
            initial_nSpikes_join = data_initial_state[(data_initial_state.paramI == param_I) & (data_initial_state.neuron >= min_range_neuron) & (data_initial_state.neuron < max_range_neuron)][["attack", "time_delta", "neuron"]].copy()
            
            attack_nSpikes_join = df_attack[(df_attack.neuron >= min_range_neuron) & (df_attack.neuron < max_range_neuron)][["attack", "time_delta", "neuron"]].copy()
            
            join_initial_attack_shifts = pd.merge(attack_nSpikes_join, initial_nSpikes_join,  how='left', left_on=['time_delta','neuron'], right_on = ['time_delta','neuron'])
            join_initial_attack_shifts = join_initial_attack_shifts.fillna(0)
            n_shifts_initial_attack = join_initial_attack_shifts[join_initial_attack_shifts.attack_y == 0].count()["neuron"]
            percent_shifts_initial_attack = round((n_shifts_initial_attack/attack_nSpikes)*100,2)

            # List of neurons shifted in this layer
            attack_shifted_neurons = join_initial_attack_shifts[join_initial_attack_shifts.attack_y == 0]["neuron"].unique()

            ### ATTACKED NEURONS PERSPECTIVE

            # spikes: mean
            a_initial_mean_spikes = round(d_initial_aggr_neuron[(d_initial_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].mean(), 2)
            a_attack_mean_spikes = round(d_attack_aggr_neuron[(d_attack_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].mean(), 2)

            # Standard deviation
            a_initial_std_spikes = round(d_initial_aggr_neuron[(d_initial_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].std(), 2)
            a_attack_std_spikes = round(d_attack_aggr_neuron[(d_attack_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].std(), 2)

            # mean of % shifts
            a_join_shifts_attk = join_initial_attack_shifts[join_initial_attack_shifts.neuron.isin(attack_shifted_neurons)]
            a_n_shifts_zeros = a_join_shifts_attk[a_join_shifts_attk.attack_y == 0].count()["neuron"]
            # number of shifts under attack (count zeros) / total number of spikes under attack 
            a_mean_shifts = round((a_n_shifts_zeros/a_join_shifts_attk.neuron.count())*100, 2)
            
            data_csv.append(initial_mean_spikes)
            data_csv.append(attack_mean_spikes)
            data_csv.append(initial_std_spikes)
            data_csv.append(attack_std_spikes)
            data_csv.append(percent_shifts_initial_attack)

            data_csv.append(a_initial_mean_spikes)
            data_csv.append(a_attack_mean_spikes)
            data_csv.append(a_initial_std_spikes)
            data_csv.append(a_attack_std_spikes)
            data_csv.append(a_mean_shifts)

        append_to_csv_file(stats_neurons_layers_csv_filename, data_csv)

In [ ]:
def generate_aggr_data_pandas(test_id, position_attack, attack, n_exec, vIncrement, n_neurons, flag, csv_filename, aggr_csv_filename):   
    df = pd.read_csv(csv_filename, delimiter=";", dtype={'n_neurons': np.int32})

    data_initial_state = df[df.attack == "initial_state"]
    df_attack = df[(df.attack == attack) & (df.vIncrement == vIncrement)] 

    # Get the data from the initial_state experiment based on the paramI value
    param_I = df_attack.paramI.unique()[0]
    
    for position in range(0, len(list_instant_optimal_path)):
        min_range = list_instant_optimal_path[position]

        if position == len(list_instant_optimal_path)-1: # last position of the list
            max_range = round(SIMULATION_TIME/ms)
        else:
            max_range = list_instant_optimal_path[position+1]

        # Dump spontaneous aggr data only if flag == True
        if flag:
            # Dump to CSV
            # Group by time instant: Extraction of time_delta and count number of spikes in each instant
            grouped = data_initial_state[(data_initial_state.paramI == param_I) & (data_initial_state.time_delta >= min_range) & (data_initial_state.time_delta < max_range)].groupby('time_delta').count()["neuron"]
            d_aggr_initial = pd.DataFrame()
            d_aggr_initial['time_delta'] = grouped.index
            d_aggr_initial['number_spikes'] = grouped.tolist() #[item/276 for item in grouped.tolist()]
            d_aggr_initial["position_attack"] = position_attack
            
            d_aggr_initial["attack"] = "initial_state"
            d_aggr_initial["position"] = int(min_range/(STEP_TIME/ms)) #Get number of position
            d_aggr_initial["n_neurons"] = 0 # Initial state does not have n_neurons attacked
            d_aggr_initial["v_increment"] = 0 # Initial state does not have v_increment attack
            d_aggr_initial["n_exec"] = n_exec
            
            # Export both DF to the same CSV file (append)
            d_aggr_initial.to_csv(aggr_csv_filename, mode = "a", index = False, header=False, sep=";")
        
        grouped = df_attack[(df_attack.paramI == param_I) & (df_attack.time_delta >= min_range) & (df_attack.time_delta < max_range)].groupby('time_delta').count()["neuron"]
        d_aggr_attack = pd.DataFrame()
        d_aggr_attack['time_delta'] = grouped.index
        d_aggr_attack['number_spikes'] = grouped.tolist() #[item/276 for item in grouped.tolist()]
        d_aggr_attack["position_attack"] = position_attack

        d_aggr_attack["attack"] = attack
        d_aggr_attack["position"] = int(min_range/(STEP_TIME/ms)) #Get number of position
        d_aggr_attack["n_neurons"] = n_neurons
        d_aggr_attack["v_increment"] = vIncrement
        d_aggr_attack["n_exec"] = n_exec

        d_aggr_attack.to_csv(aggr_csv_filename, mode = "a", index = False, header=False, sep=";")

#### FLO

In [ ]:
if "FLO" in list_attack_generation:
    currTest = 1
    stimValue = 1.0
    paramI = 10
    #currExec = 0
    nNeurons = 100
    position_attack = 0
    param_I = 0

    spontaneous_params = {
        "attack": "initial_state"
    }

    spikeMon, stateMon = neuronal_simulation(SIMULATION_TIME, spontaneous_params, {})

    flag_initial = True
    for currExec in range(0, 10):
        print(currExec)

        FLO_params = {
            "attack": "FLO",
            "instants_attack": [10*ms],
            "neuron_list": generate_list_random_neurons(nNeurons),
            "vIncrement": 40,
        }

        FLO_results_simulation = {}

        spikeMon_a, stateMon_a = neuronal_simulation(SIMULATION_TIME, FLO_params, FLO_results_simulation)

        dump_simulation_data_to_csv("initial_state", str(currTest), str(position_attack), "0", "0", "X", "X", "0", "0", "0", str(paramI), spikeMon, csv_filename_FLO) 
        dump_simulation_data_to_csv("FLO", str(currTest), str(position_attack), str(len(FLO_params["instants_attack"])), str(len(FLO_params["neuron_list"])), list_neurons_to_string(FLO_params["neuron_list"]), "(0,0)", str(stimValue), str(currExec), str(FLO_params["vIncrement"]), str(paramI), spikeMon_a, csv_filename_FLO)

        generate_aggr_data_pandas(currTest, position_attack, "FLO", currExec, FLO_params["vIncrement"], nNeurons, flag_initial, csv_filename_FLO, aggr_csv_filename_FLO)

        if flag_initial:
            flag_initial = False

        generate_stats_neurons_pandas(currTest, position_attack, "FLO", csv_filename_FLO, stats_neurons_global_csv_filename_FLO, stats_neurons_layers_csv_filename_FLO)    

        # Overwrite export file for next execution
        open(csv_filename_FLO, 'w').close()
        append_to_csv_file(csv_filename_FLO, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])

In [ ]:
df_total_spikes_FLO = pd.read_csv(aggr_csv_filename_FLO, delimiter=";")
df_total_spikes_FLO = df_total_spikes_FLO.drop(['position_attack'], axis = 1)
df_total_spikes_FLO = pd.DataFrame(df_total_spikes_FLO.groupby(["attack", "position", "n_exec"])["number_spikes"].sum())
df_total_spikes_FLO.reset_index(inplace=True)

In [ ]:
df_total_spikes_FLO.to_csv(stats_dir_FLO+'total_spikes.csv', index=False, sep=";")

#### JAM

In [ ]:
if "JAM" in list_attack_generation:
    currTest = 1
    stimValue = 1.0
    vIncrement = 0
    paramI = 10
    #currExec = 0
    nNeurons = 100
    position_attack = 0

    spontaneous_params = {
        "attack": "initial_state"
    }

    spikeMon, stateMon = neuronal_simulation(SIMULATION_TIME, spontaneous_params, {})

    flag_initial = True
    for currExec in range(0, 10):
        print(currExec)

        JAM_params = {
            "attack": "JAM",
            "init_attack": 10*ms,
            "end_attack": 60*ms,
            "neuron_list": generate_list_random_neurons(nNeurons),
        }

        JAM_results_simulation = {}

        spikeMon_a, stateMon_a = neuronal_simulation(SIMULATION_TIME, JAM_params, JAM_results_simulation)

        dump_simulation_data_to_csv("initial_state", str(currTest), str(position_attack), "0", "0", "X", "X", "0", "0", "0", str(paramI), spikeMon, csv_filename_JAM) 
        dump_simulation_data_to_csv("JAM", str(currTest), str(position_attack), "1", str(len(JAM_params["neuron_list"])), list_neurons_to_string(JAM_params["neuron_list"]), "(0,0)", str(stimValue), str(currExec), str(vIncrement), str(paramI), spikeMon_a, csv_filename_JAM)

        generate_aggr_data_pandas(currTest, position_attack, "JAM", currExec, vIncrement, nNeurons, flag_initial, csv_filename_JAM, aggr_csv_filename_JAM)

        if flag_initial:
            flag_initial = False

        generate_stats_neurons_pandas(currTest, position_attack, "JAM", csv_filename_JAM, stats_neurons_global_csv_filename_JAM, stats_neurons_layers_csv_filename_JAM)    

        # Overwrite export file for next execution
        open(csv_filename_JAM, 'w').close()
        append_to_csv_file(csv_filename_JAM, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])

In [ ]:
df_total_spikes_JAM = pd.read_csv(aggr_csv_filename_JAM, delimiter=";")
df_total_spikes_JAM = df_total_spikes_JAM.drop(['position_attack'], axis = 1)
df_total_spikes_JAM = pd.DataFrame(df_total_spikes_JAM.groupby(["attack", "position", "n_exec"])["number_spikes"].sum())
df_total_spikes_JAM.reset_index(inplace=True)

In [ ]:
df_total_spikes_JAM.to_csv(stats_dir_JAM+'total_spikes.csv', index=False, sep=";")

#### SCA

In [ ]:
if "SCA" in list_attack_generation:
    currTest = 1
    stimValue = 1.0
    paramI = 10
    currExec = 0
    nNeurons = 200
    position_attack = 0

    spontaneous_params = {
        "attack": "initial_state"
    }

    spikeMon, stateMon = neuronal_simulation(SIMULATION_TIME, spontaneous_params, {})

    SCA_params = {
        "attack": "SCA",
        "init_attack": 10*ms,
        "steps_attack": get_time_steps_sequential(10*ms),
        "vIncrement": 40,
    }

    SCA_results_simulation = {}

    spikeMon_a, stateMon_a = neuronal_simulation(SIMULATION_TIME, SCA_params, SCA_results_simulation)

    dump_simulation_data_to_csv("initial_state", str(currTest), str(position_attack), "0", "0", "X", "X", "0", "0", "0", str(paramI), spikeMon, csv_filename_SCA) 
    dump_simulation_data_to_csv("SCA", str(currTest), str(position_attack), "1", str(200), "X", "(0,0)", str(stimValue), str(currExec), str(SCA_params["vIncrement"]), str(paramI), spikeMon_a, csv_filename_SCA)

    generate_aggr_data_pandas(currTest, position_attack, "SCA", currExec, SCA_params["vIncrement"], nNeurons, True, csv_filename_SCA, aggr_csv_filename_SCA)
    generate_stats_neurons_pandas(currTest, position_attack, "SCA", csv_filename_SCA, stats_neurons_global_csv_filename_SCA, stats_neurons_layers_csv_filename_SCA)    

    # Overwrite export file for next execution
    open(csv_filename_SCA, 'w').close()
    append_to_csv_file(csv_filename_SCA, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])

In [ ]:
df_total_spikes_SCA = pd.read_csv(aggr_csv_filename_SCA, delimiter=";")
df_total_spikes_SCA = df_total_spikes_SCA.drop(['position_attack', 'n_exec'], axis = 1)
df_total_spikes_SCA = pd.DataFrame(df_total_spikes_SCA.groupby(["attack", "position"])["number_spikes"].sum())
df_total_spikes_SCA.reset_index(inplace=True)
df_total_spikes_SCA

In [ ]:
df_total_spikes_SCA.to_csv(stats_dir_SCA+'total_spikes.csv', index=False, sep=";")

#### FOR

In [ ]:
if "FOR" in list_attack_generation:
    currTest = 1
    stimValue = 1.0
    paramI = 10
    currExec = 0
    nNeurons = 200
    position_attack = 0

    spontaneous_params = {
        "attack": "initial_state"
    }

    spikeMon, stateMon = neuronal_simulation(SIMULATION_TIME, spontaneous_params, {})

    FOR_params = {
        "attack": "FOR",
        "init_attack": 10*ms,
        "steps_attack": get_time_steps_sequential(10*ms),
        "vIncrement": 40,
    }

    FOR_results_simulation = {}

    spikeMon_a, stateMon_a = neuronal_simulation(SIMULATION_TIME, FOR_params, FOR_results_simulation)

    dump_simulation_data_to_csv("initial_state", str(currTest), str(position_attack), "0", "0", "X", "X", "0", "0", "0", str(paramI), spikeMon, csv_filename_FOR) 
    dump_simulation_data_to_csv("FOR", str(currTest), str(position_attack), "1", str(200), "X", "(0,0)", str(stimValue), str(currExec), str(FOR_params["vIncrement"]), str(paramI), spikeMon_a, csv_filename_FOR)

    generate_aggr_data_pandas(currTest, position_attack, "FOR", currExec, FOR_params["vIncrement"], nNeurons, True, csv_filename_FOR, aggr_csv_filename_FOR)
    generate_stats_neurons_pandas(currTest, position_attack, "FOR", csv_filename_FOR, stats_neurons_global_csv_filename_FOR, stats_neurons_layers_csv_filename_FOR)    

    # Overwrite export file for next execution
    open(csv_filename_FOR, 'w').close()
    append_to_csv_file(csv_filename_FOR, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])

In [ ]:
df_total_spikes_FOR = pd.read_csv(aggr_csv_filename_FOR, delimiter=";")
df_total_spikes_FOR = df_total_spikes_FOR.drop(['position_attack', 'n_exec'], axis = 1)
df_total_spikes_FOR = pd.DataFrame(df_total_spikes_FOR.groupby(["attack", "position"])["number_spikes"].sum())
df_total_spikes_FOR.reset_index(inplace=True)
df_total_spikes_FOR

In [ ]:
df_total_spikes_FOR.to_csv(stats_dir_FOR+'total_spikes.csv', index=False, sep=";")

#### SPO

In [ ]:
if "SPO" in list_attack_generation:
    currTest = 1
    stimValue = 1.0
    vIncrement = 0
    paramI = 10
    currExec = 0
    nNeurons = 200
    position_attack = 0

    spontaneous_params = {
        "attack": "initial_state"
    }

    spikeMon, stateMon = neuronal_simulation(SIMULATION_TIME, spontaneous_params, {})

    SPO_params = {
        "attack": "SPO",
        "neuron_list_recorded": list(range(0, 100,1)),
        "neuron_list_attack": list(range(100,200,1)),
        "init_record": 10*ms,
        "end_record": 60*ms,
        "init_attack": 70*ms,
        "end_attack": 120*ms,
        "recorded_activity": {}
    }

    # Extract legitimate neuronal activity (record) to be replicated by the SPO attack
    init_record_formatted = round(SPO_params["init_record"]/ms*10)
    end_record_formatted = round(SPO_params["end_record"]/ms*10)

    # Extend SPO_params with the recorded activity from the spontaneous behavior for the targeted neurons
    neuron_counter = 0
    for neuron in SPO_params["neuron_list_recorded"]:
        SPO_params["recorded_activity"][SPO_params["neuron_list_attack"][neuron_counter]] = list(stateMon.v[neuron][init_record_formatted: end_record_formatted])
        neuron_counter += 1

    spikeMon_a, stateMon_a = neuronal_simulation(SIMULATION_TIME, SPO_params, {})
    
    dump_simulation_data_to_csv("initial_state", str(currTest), str(position_attack), "0", "0", "X", "X", "0", "0", "0", str(paramI), spikeMon, csv_filename_SPO) 
    dump_simulation_data_to_csv("SPO", str(currTest), str(position_attack), "1", str(200), "X", "(0,0)", str(stimValue), str(currExec), str(vIncrement), str(paramI), spikeMon_a, csv_filename_SPO)

    generate_aggr_data_pandas(currTest, position_attack, "SPO", currExec, vIncrement, nNeurons, True, csv_filename_SPO, aggr_csv_filename_SPO)
    generate_stats_neurons_pandas(currTest, position_attack, "SPO", csv_filename_SPO, stats_neurons_global_csv_filename_SPO, stats_neurons_layers_csv_filename_SPO)    

    # Overwrite export file for next execution
    open(csv_filename_SPO, 'w').close()
    append_to_csv_file(csv_filename_SPO, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])

In [ ]:
df_total_spikes_SPO = pd.read_csv(aggr_csv_filename_SPO, delimiter=";")
df_total_spikes_SPO = df_total_spikes_SPO.drop(['position_attack', 'n_exec'], axis = 1)
df_total_spikes_SPO = pd.DataFrame(df_total_spikes_SPO.groupby(["attack", "position"])["number_spikes"].sum())
df_total_spikes_SPO.reset_index(inplace=True)
df_total_spikes_SPO

In [ ]:
df_total_spikes_SPO.to_csv(stats_dir_SPO+'total_spikes.csv', index=False, sep=";")

In [ ]:
df_total_spikes_SPO

#### SYB

In [ ]:
if "SYB" in list_attack_generation:
    currTest = 1
    stimValue = 1.0
    vIncrement = 0
    paramI = 10
    #currExec = 0
    nNeurons = 100
    position_attack = 0
    param_I = 0

    spontaneous_params = {
        "attack": "initial_state"
    }

    spikeMon, stateMon = neuronal_simulation(SIMULATION_TIME, spontaneous_params, {})

    flag_initial = True

    for currExec in range(0, 10):
        print(currExec)
        SYB_params = {
            "attack": "SYB",
            "instants_attack": [10*ms],
            "neuron_list": generate_list_random_neurons(nNeurons),
        }

        SYB_results_simulation = {
            "stimulation": {}, 
            "inhibition": {},
        }

        spikeMon_a, stateMon_a = neuronal_simulation(SIMULATION_TIME, SYB_params, SYB_results_simulation)

        dump_simulation_data_to_csv("initial_state", str(currTest), str(position_attack), "0", "0", "X", "X", "0", "0", "0", str(paramI), spikeMon, csv_filename_SYB) 
        dump_simulation_data_to_csv("SYB", str(currTest), str(position_attack), str(len(SYB_params["instants_attack"])), str(len(SYB_params["neuron_list"])), list_neurons_to_string(SYB_params["neuron_list"]), "(0,0)", str(stimValue), str(currExec), str(vIncrement), str(paramI), spikeMon_a, csv_filename_SYB)

        generate_aggr_data_pandas(currTest, position_attack, "SYB", currExec, vIncrement, nNeurons, flag_initial, csv_filename_SYB, aggr_csv_filename_SYB)

        if flag_initial:
            flag_initial = False

        generate_stats_neurons_pandas(currTest, position_attack, "SYB", csv_filename_SYB, stats_neurons_global_csv_filename_SYB, stats_neurons_layers_csv_filename_SYB)    

        # Overwrite export file for next execution
        open(csv_filename_SYB, 'w').close()
        append_to_csv_file(csv_filename_SYB, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])

In [ ]:
df_total_spikes_SYB = pd.read_csv(aggr_csv_filename_SYB, delimiter=";")
df_total_spikes_SYB = df_total_spikes_SYB.drop(['position_attack'], axis = 1)
df_total_spikes_SYB = pd.DataFrame(df_total_spikes_SYB.groupby(["attack", "position", "n_exec"])["number_spikes"].sum())
df_total_spikes_SYB.reset_index(inplace=True)

In [ ]:
df_total_spikes_SYB.to_csv(stats_dir_SYB+'total_spikes.csv', index=False, sep=";")

In [ ]:
df_total_spikes_SYB

#### SIN

In [ ]:
if "SIN" in list_attack_generation:
    currTest = 1
    stimValue = 1.0
    paramI = 10
    currExec = 0
    nNeurons = 100
    position_attack = 0
    param_I = 0

    spontaneous_params = {
        "attack": "initial_state"
    }

    spikeMon, stateMon = neuronal_simulation(SIMULATION_TIME, spontaneous_params, {})

    SIN_params = {
        "attack": "SIN",
        "instants_attack": [10*ms],
        "vIncrement": 40,
        "target_neuron_2nd_layer": 0,
    }

    SIN_results_simulation = {}

    # Obtain the list of neurons connected to the target neuron from the 2nd layer -> attacked neurons
    df_neurons = pd.read_csv("synapsysConv1-Conv2.csv", delimiter=";", usecols=["sourceNeuron", "targetNeuron"])
    SIN_params["neuron_list"] = df_neurons[df_neurons.targetNeuron == SIN_params["target_neuron_2nd_layer"]].sourceNeuron.unique().tolist()

    spikeMon_a, stateMon_a = neuronal_simulation(SIMULATION_TIME, SIN_params, SIN_results_simulation)

    dump_simulation_data_to_csv("initial_state", str(currTest), str(position_attack), "0", "0", "X", "X", "0", "0", "0", str(paramI), spikeMon, csv_filename_SIN) 
    dump_simulation_data_to_csv("SIN", str(currTest), str(position_attack), str(1), str(len(SIN_params["neuron_list"])), list_neurons_to_string(SIN_params["neuron_list"]), "(0,0)", str(stimValue), str(currExec), str(SIN_params["vIncrement"]), str(paramI), spikeMon_a, csv_filename_SIN)

    generate_aggr_data_pandas(currTest, position_attack, "SIN", currExec, SIN_params["vIncrement"], nNeurons, True, csv_filename_SIN, aggr_csv_filename_SIN)
    generate_stats_neurons_pandas(currTest, position_attack, "SIN", csv_filename_SIN, stats_neurons_global_csv_filename_SIN, stats_neurons_layers_csv_filename_SIN)    

    # Overwrite export file for next execution
    open(csv_filename_SIN, 'w').close()
    append_to_csv_file(csv_filename_SIN, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])

In [ ]:
df_total_spikes_SIN = pd.read_csv(aggr_csv_filename_SIN, delimiter=";")
df_total_spikes_SIN = df_total_spikes_SIN.drop(['position_attack', 'n_exec'], axis = 1)
df_total_spikes_SIN = pd.DataFrame(df_total_spikes_SIN.groupby(["attack", "position"])["number_spikes"].sum())
df_total_spikes_SIN.reset_index(inplace=True)

In [ ]:
df_total_spikes_SIN.to_csv(stats_dir_SIN+'total_spikes.csv', index=False, sep=";")

#### NON

In [ ]:
if "NON" in list_attack_generation:
    currTest = 1
    stimValue = 1.0
    currExec = 0
    nNeurons = 105
    vIncrement = 0
    position_attack = 0
    param_I = 0

    spontaneous_params = {
        "attack": "initial_state"
    }

    spikeMon, stateMon = neuronal_simulation(SIMULATION_TIME, spontaneous_params, {})

    NON_params = {
        "attack": "NON",
        "init_attack": 10*ms,
        "steps_attack": get_time_steps_sequential(10*ms),
        "neuron_list": list(range(0,200,1)), # All neurons in the first layer
        "vIncrement": 40,
        "probability_spontaneous": 60,
        "probability_stimulation": 20,
        "probability_inhibition": 20,
    }

    NON_results_simulation = {
        "stimulation": {}, 
        "inhibition": {},
    }

    spikeMon_a, stateMon_a = neuronal_simulation(SIMULATION_TIME, NON_params, NON_results_simulation)

    dump_simulation_data_to_csv("initial_state", str(currTest), str(position_attack), "0", "0", "X", "X", "0", "0", "0", str(paramI), spikeMon, csv_filename_NON) 
    dump_simulation_data_to_csv("NON", str(currTest), str(position_attack), str(1), str(len(NON_params["neuron_list"])), list_neurons_to_string(NON_params["neuron_list"]), "(0,0)", str(stimValue), str(currExec), str(NON_params["vIncrement"]), str(paramI), spikeMon_a, csv_filename_NON)

    generate_aggr_data_pandas(currTest, position_attack, "NON", currExec, NON_params["vIncrement"], nNeurons, True, csv_filename_NON, aggr_csv_filename_NON)
    generate_stats_neurons_pandas(currTest, position_attack, "NON", csv_filename_NON, stats_neurons_global_csv_filename_NON, stats_neurons_layers_csv_filename_NON)    

    # Overwrite export file for next execution
    open(csv_filename_NON, 'w').close()
    append_to_csv_file(csv_filename_NON, ["attack", "test", "position_attack", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])

In [ ]:
df_total_spikes_NON = pd.read_csv(aggr_csv_filename_NON, delimiter=";")
df_total_spikes_NON = df_total_spikes_NON.drop(['position_attack', 'n_exec'], axis = 1)
df_total_spikes_NON = pd.DataFrame(df_total_spikes_NON.groupby(["attack", "position"])["number_spikes"].sum())
df_total_spikes_NON.reset_index(inplace=True)

In [ ]:
df_total_spikes_NON.to_csv(stats_dir_NON+'total_spikes.csv', index=False, sep=";")

## Generate plot for results

In [ ]:
plt.rcParams.update({'font.family': 'monospace'})
sns.set(style="whitegrid",font_scale=2.5, rc={'figure.figsize':(40,20)})

#kwargs  =   {'edgecolor':"black", # for edge color
#             'linewidth':2, # line width of spot
#             'linestyle':'-', # line style of spot
#            }

plt.rcParams['axes.edgecolor'] = "black" #set the value globally
sns.set_palette("colorblind")
#sns.set(font_scale=1.4)
#sns.set_style("whitegrid")

In [ ]:
df_FLO = pd.read_csv(stats_dir_FLO+'total_spikes.csv', delimiter=";")
df_JAM = pd.read_csv(stats_dir_JAM+'total_spikes.csv', delimiter=";")
df_SCA = pd.read_csv(stats_dir_SCA+'total_spikes.csv', delimiter=";")
df_FOR = pd.read_csv(stats_dir_FOR+'total_spikes.csv', delimiter=";")
df_SPO = pd.read_csv(stats_dir_SPO+'total_spikes.csv', delimiter=";")
df_SYB = pd.read_csv(stats_dir_SYB+'total_spikes.csv', delimiter=";")
df_SIN = pd.read_csv(stats_dir_SIN+'total_spikes.csv', delimiter=";")
df_NON = pd.read_csv(stats_dir_NON+'total_spikes.csv', delimiter=";")

df_SCA["n_exec"] = 0
df_FOR["n_exec"] = 0
df_SPO["n_exec"] = 0
df_SIN["n_exec"] = 0
df_NON["n_exec"] = 0

In [ ]:
df_spikes = pd.concat([df_FLO, df_JAM[df_JAM.attack == "JAM"], 
                    df_SCA[df_SCA.attack == "SCA"], df_FOR[df_FOR.attack == "FOR"], 
                    df_SPO[df_SPO.attack == "SPO"], df_SYB[df_SYB.attack == "SYB"], 
                    df_SIN[df_SIN.attack == "SIN"], df_NON[df_NON.attack == "NON"]])

# Rename "initial_state" to "Spontaneous"
df_spikes.loc[df_spikes['attack']=="initial_state",'attack'] = 'Spontaneous'
df_spikes.position += 1

In [ ]:
## DF CONTAINING PERCENT CHANGES FOR EACH ATTACK, POSITION AND N_EXEC, READY TO PLOT ##

df_percentages = pd.DataFrame()

# (v2 - v2)/|v1| x 100

# For those attacks with multiple executions, we calculate the percentage change for each execution.
for attack in list_attack_generation:   
    for position in range(1, 6): 
        v1 = df_spikes[(df_spikes.attack == "Spontaneous") & (df_spikes.position == position)].mean()["number_spikes"]
            
        if attack in ["FLO", "JAM", "SYB"]:
            v2_list = df_spikes[(df_spikes.attack == attack) & (df_spikes.position == position)]["number_spikes"]
            
            counter = 1
            for v2_value in v2_list:
                percent_change = (v2_value-v1)/abs(v1) * 100
                
                data = {
                    "attack": [attack],
                    "position": [position],
                    "exec": [counter],
                    "percentage": [round(percent_change, 2)]
                }

                df_row = pd.DataFrame(data)
                df_percentages = pd.concat([df_percentages, df_row])
                
                counter += 1
                
        else:
            v2 = df_spikes[(df_spikes.attack == attack) & (df_spikes.position == position)].mean()["number_spikes"]

            percent_change = (v2-v1)/abs(v1) * 100
            
            data = {
                "attack": [attack],
                "position": [position],
                "exec": [counter],
                "percentage": [round(percent_change, 2)]
            }

            df_row = pd.DataFrame(data)
            df_percentages = pd.concat([df_percentages, df_row])
    
    for position in range(23, 28): 
        v1 = df_spikes[(df_spikes.attack == "Spontaneous") & (df_spikes.position == position)].mean()["number_spikes"]
            
        if attack in ["FLO", "JAM", "SYB"]:
            v2_list = df_spikes[(df_spikes.attack == attack) & (df_spikes.position == position)]["number_spikes"]
            
            counter = 1
            for v2_value in v2_list:
                percent_change = (v2_value-v1)/abs(v1) * 100
                #percentages[attack+"_final"].append(round(percent_change, 2))
                
                data = {
                    "attack": [attack],
                    "position": [position],
                    "exec": [counter],
                    "percentage": [round(percent_change, 2)]
                }

                df_row = pd.DataFrame(data)
                df_percentages = pd.concat([df_percentages, df_row])
                
                counter += 1
        else:
            v2 = df_spikes[(df_spikes.attack == attack) & (df_spikes.position == position)].mean()["number_spikes"]

            percent_change = (v2-v1)/abs(v1) * 100
            
            data = {
                "attack": [attack],
                "position": [position],
                "exec": [counter],
                "percentage": [round(percent_change, 2)]
            }

            df_row = pd.DataFrame(data)
            df_percentages = pd.concat([df_percentages, df_row])
            
# Multiply by -1 to obtain the decrease % in a positive value (instead of -x%)    
df_percentages["percentage"] *= -1

In [ ]:
# Add new column to indicate the group of positions
df_first_group = df_percentages[df_percentages.position.isin([1,2,3,4,5])].copy()
df_first_group["Window"] = "First five positions"

df_spikes_plot = df_first_group.copy()

df_last_group = df_percentages[df_percentages.position.isin([23,24,25,26,27])].copy()
df_last_group["Window"] = "Last five positions"

df_spikes_plot = pd.concat([df_spikes_plot, df_last_group])

In [ ]:
#fig, ax = plt.subplots(figsize=(40,20))

g = sns.catplot(x="attack", y="percentage", col="Window", data=df_spikes_plot, kind="bar", height=10, aspect=2)

label_size = 45
ticks_size = 35
legend_size = 35
numbers_size = 35

ax = g.facet_axis(0,0)
ax.set_xlabel("Attack", fontsize=label_size, fontweight="bold")
ax.set_ylabel("Decrease percentage", fontsize=label_size, fontweight="bold")
ax.tick_params(labelsize=ticks_size)

ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)

ax = g.facet_axis(0,1)
ax.set_xlabel("Attack", fontsize=label_size, fontweight="bold")
ax.tick_params(labelsize=ticks_size)

ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)

ax = g.facet_axis(0,0)

counter = 1

distances_initial = {
    1: 50,
    2: 45, 
    3: 60, 
    4: 45,
    5: 90, 
    6: 50, 
    7: 60,
    8: 135, 
}


counter = 1
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.2f'), 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha = 'center', va = 'center', 
                    size=numbers_size,
                    xytext = (0, distances_initial[counter]), 
                    textcoords = 'offset points')
    counter+=1

ax = g.facet_axis(0,1)

distances_final = {
    1: 40,
    2: 40, 
    3: 50, 
    4: 30,
    5: 70, 
    6: 40, 
    7: 60,
    8: 80, 
}

counter = 1
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.2f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   size=numbers_size,
                   xytext = (0, distances_final[counter]), 
                   textcoords = 'offset points')
    counter+=1

#plt.savefig("results.pdf", bbox_inches='tight')
#plt.close()